# TravelTide Customer Segmentation
TravelTide dataset has been filtered by cohort on SQL and we will now import it for cleaning, aggregation and clustering

## Steps for Analysis

1. **Data Exploration**: Understand the structure and content of the dataset.
2. **Data Cleaning**: Prepare the data for behavioural analysis.
3. **Feature Engineering and Aggregations**: Summarize the data at the user-level, creating new metrics as needed.
4. **Data Segmentation**: Cluster customers based on their behavior.
5. **Perk Allocation**: Allocate users to their assigned perks.
6. **Validation and Evaluation**: Run statistical tests to validate the clusters and evaluate the effectiveness of the perk allocation.
7. **Conclusion**: Write a report summarizing findings and suggesting next steps.

# 1. Data Import
I'll start by importing the dataset 'cohort_joined.csv' into a DataFrame called 'cohort_df'.

In [ ]:
# Importing the Dataset
import pandas as pd

# Load the dataset into a DataFrame named 'traveltide_df'
cohort_df = pd.read_csv('cohort_joined.csv')

# Display the first few rows of the DataFrame
cohort_df.head()

In [ ]:
cohort_df.shape

In [ ]:
cohort_df.describe()

# 2. Data Cleaning
I'll explore the dataset and clean where necessary following the below steps:

1. **Convert Datestamps to Datetime**: Ensure all datestamp columns are in datetime format for easier manipulation.
2. **Handle Missing Values**: Fill in or remove missing values based on the nature of each variable.
3. **Handle Outliers**: Identify and handle outliers in numerical features to avoid skewing the data.
4. **Aggregate by Users**: Summarize the data by `user_id` to get a holistic view of each customer's behavior.

### 2.1 Convert Datestamps to Datetime
I must convert all datestamp columns to datetime format for easier manipulation.

In [ ]:
# Convert Datestamps to Datetime

# Identify columns with datestamp information and include 'session_start' and 'session_end'
date_columns = [col for col in cohort_df.columns if 'date' in col.lower() or 'time' in col.lower() or col in ['session_start', 'session_end']]

# Convert these columns to datetime format
cohort_df[date_columns] = cohort_df[date_columns].apply(pd.to_datetime)

# Check the data types to confirm
cohort_df[date_columns].dtypes


### 2.2 Handle Missing Values
In this step, I'll identify and handle missing values in the dataset, acccording to their data types and my domain knowledge, always considering the impact to unsupervised learning algorithms

In [ ]:
# Identify columns with missing values
missing_data = cohort_df.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data

I'll handle the missing values in the following manner:
1. **Set to Mean**: For columns like `flight_discount_amount` and `hotel_discount_amount`, I'll set missing values to the mean. This indicates that had they booked they would have behaved averagely.
2. **Replace with 0**: For numerical columns like `seats`, `return_flight_booked`, etc., I'll replace missing values with zero.
3. **Placeholder 'Unknown'**: For categorical columns like `trip_id`, `destination_airport`, etc., I'll replace missing values with the placeholder 'Unknown'.
4. **Time-based Placeholder**: For time-based columns like `departure_time`, `return_time`, etc., I'll use a time-based placeholder.

In [ ]:
# First, let's copy the original DataFrame to a new one for cleaning
cleaned_df = cohort_df.copy()

# Group 1: Set to Mean
for col in ['flight_discount_amount', 'hotel_discount_amount']:
    mean_value = cleaned_df[col].mean()
    cleaned_df[col].fillna(mean_value, inplace=True)

# Group 2: Replace with 0
for col in ['seats', 'return_flight_booked', 'checked_bags', 'base_fare_usd', 'nights', 'rooms', 'hotel_per_room_usd']:
    cleaned_df[col].fillna(0, inplace=True)

# Group 3: Placeholder 'Unknown'
for col in ['trip_id', 'origin_airport', 'destination', 'destination_airport', 'trip_airline', 'hotel_name', 'destination_airport_lat', 'destination_airport_lon']:
    cleaned_df[col].fillna('Unknown', inplace=True)

# Group 4: Time-based Placeholder
for col in ['departure_time', 'return_time', 'check_in_time', 'check_out_time']:
    cleaned_df[col].fillna(pd.Timestamp('1900-01-01 00:00:00'), inplace=True)

# Verify if all missing values have been handled
cleaned_df.isnull().sum().sum()

### 2.3 Handle Outliers
In this step, I'll identify and handle outliers in the numerical features of the dataset. Outliers can skew the data and affect the results of our analysis. We'll use statistical methods to detect and manage these extreme values.

In [ ]:
# Identifying outliers using the IQR method:

import matplotlib.pyplot as plt
import seaborn as sns


outliers = {}
for col in cohort_df.select_dtypes(include=['float64', 'int64']).columns:  # searching for numeric columns
    Q1 = cohort_df[col].quantile(0.25)
    Q3 = cohort_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = ((cohort_df[col] < lower_bound) | (cohort_df[col] > upper_bound)).sum()
    if outlier_count > 0:
        outliers[col] = outlier_count

outliers

**Visualizing outliers:**
when analysing user behaviour, a deeper unsderstanding of the nature of the data is required:
Outliers in flight_discount_amount might be legitimate special offers, while extreme values in checked_bags might be errors. Rooms likely don't require any handling, etc... Airport coordinates are obviously not considered.

Let´s have a look at these outliers:

In [ ]:
import matplotlib.pyplot as plt

# Creating a list to display

columns_to_plot = ['seats', 'checked_bags', 'base_fare_usd', 'flight_discount_amount', 'hotel_discount_amount', 'page_clicks', 'nights', 'rooms', 'hotel_per_room_usd']

# Create boxplots for each string (column) in the list
plt.figure(figsize=(10, 6))
for i, col in enumerate(columns_to_plot):
    plt.subplot(3, 3, i+1)
    plt.boxplot(cleaned_df[col].dropna(), vert=False)
    plt.title(col)
    plt.xlabel('Value')

plt.tight_layout()
plt.show()

**Findings and Approach for Handling Outliers**
- **Seats**: short range as expected
- **Checked Bags**: short range as expected
- **Base Fare USD**: Some high-fare values are identified as outliers.
- **Flight Discount Amount & Hotel Discount Amount**: Most values are around 0, with outliers representing higher discount amounts.
- **Page Clicks**: Outliers at higher values may represent users with extensive browsing behaviour.
- **Nights**: Most values are clustered within a standard range for hotel stays, with outliers for longer stays.
- **Rooms**: Outliers at higher values will represent bookings for multiple rooms.
- **Hotel Per Room USD**: Some higher per-room rates are identified as outliers.

***Approach:***
1. Preserve outliers in domain-specific columns: `flight_discount_amount`, `hotel_discount_amount`, `seats`, `rooms`, `checked_bags`.
2. Cap outliers at the 0.01 and 0.99 quantiles for remaining selected columns.

In [ ]:
# Correct the 'nights' column by taking the absolute value
cleaned_df['nights'] = cleaned_df['nights'].abs()

# Columns to preserve outliers
preserve_outliers = ['flight_discount_amount', 'hotel_discount_amount', 'seats', 'rooms', 'checked_bags']

# Columns to cap outliers
cap_outliers = [col for col in columns_to_plot if col not in preserve_outliers]

# Cap outliers at the 0.01 and 0.99 quantiles for selected columns
for col in cap_outliers:
    lower_cap = cleaned_df[col].quantile(0.01)
    upper_cap = cleaned_df[col].quantile(0.99)
    cleaned_df[col] = cleaned_df[col].clip(lower=lower_cap, upper=upper_cap)

# Confirm that the capping has been done correctly by plotting boxplots again
plt.figure(figsize=(10, 6))
for i, col in enumerate(columns_to_plot):
    plt.subplot(3, 3, i+1)
    plt.boxplot(cleaned_df[col].dropna(), vert=False)
    plt.title(col)
    plt.xlabel('Value')

plt.tight_layout()
plt.show()


In [ ]:
cleaned_df.to_csv('cleaned_df.csv', index=False)
metrics_df = cleaned_df.copy()

# 3. Feature Engineering and Aggregation


**Calculate new Metrics** (to either use in aggregation or just meet the advanced tasks of the project assignment)

To offer further insights into user behaviour, I will calculate:
- Imporoved Distance Calculation  
- Total combined bookings
- Age

After aggregation:
- Proportion of discounted flights booked
- Proportion of discounted hotels booked
- Scaled Average Dollars Saved per km
- Bargain Hunter Index

**Improved Distance Calculation**

I will use the inversed Vicenty formula, aka Karney's Method, applied to the data set using the geodesic script in geopy library. I must account for the fact that I used placeholders when handling missing values.

In [ ]:
!pip install geopy
from geopy.distance import geodesic

# Function to calculate the distance using geodesic (Karney's Method)
def calculate_geodesic_distance(row):
    coords_1 = (row['home_airport_lat'], row['home_airport_lon'])
    coords_2 = (row['destination_airport_lat'], row['destination_airport_lon'])
    
    # Check for missing or placeholder values in coordinates
    if 'Unknown' in coords_1 or 'Unknown' in coords_2 or \
        any(pd.isna(val) for val in coords_1) or any(pd.isna(val) for val in coords_2):
        return None
    
    return geodesic(coords_1, coords_2).kilometers

# Apply the function to each row in metrics_df
metrics_df['geodesic_distance_km'] = metrics_df.apply(calculate_geodesic_distance, axis=1)


**Total Combined Bookings**

A count of all sessions that had both flights and hotel booked

In [ ]:
metrics_df['combined_bookings'] = (metrics_df['flight_booked'] & metrics_df['hotel_booked'] & ~metrics_df['cancellation']).astype(int)

**Age**

**Total Combined Discounts**

A count of all sessions that had both flights and hotel booked

In [ ]:
from datetime import datetime

# Extracting the birth year and calculating age
current_year = datetime.now().year
metrics_df['age'] = current_year - metrics_df['birthdate'].dt.year

In [ ]:
metrics_df['combined_discounts'] = (metrics_df['flight_discount'] & metrics_df['hotel_discount'] & ~metrics_df['cancellation']).astype(int)

In [ ]:
# Calculate Basic Metrics and Aggregate Data by user_id
import numpy as np

# Replace 'Unknown' with np.nan for numerical conversion
metrics_df.replace('Unknown', np.nan, inplace=True)

# Convert columns to appropriate data types to avoid type errors
metrics_df['page_clicks'] = metrics_df['page_clicks'].astype(float)
metrics_df['return_flight_booked'] = metrics_df['return_flight_booked'].astype(float)
metrics_df['hotel_per_room_usd'] = metrics_df['hotel_per_room_usd'].astype(float)

# Aggregate by user_id
basicagg_df = metrics_df.groupby('user_id').agg({
    'age': 'max',
    'session_id': 'count',
    'page_clicks': 'mean',
    'flight_booked': 'sum',
    'hotel_booked': 'sum',
    'flight_discount': 'sum',
    'flight_discount_amount': 'mean',
    'hotel_discount': 'sum',
    'hotel_discount_amount': 'mean',
    'cancellation': 'sum',
    'seats': 'mean',
    'checked_bags': 'mean',
    'return_flight_booked': 'sum',
    'rooms': 'mean',
    'nights': 'mean',
    'hotel_per_room_usd': 'mean',
    'geodesic_distance_km': 'mean',
    'combined_bookings': 'sum',
    'combined_discounts': 'sum'
}).reset_index()

# Rename columns for better understanding
basicagg_df.rename(columns={
    'session_id': 'total_sessions',
    'page_clicks': 'avg_clicks_per_session',
    'session_duration': 'avg_session_duration_mins',
    'flight_booked': 'total_flights_booked',
    'hotel_booked': 'total_hotels_booked',
    'flight_discount': 'total_flight_discounts',
    'flight_discount_amount': 'avg_flight_discount',
    'hotel_discount': 'total_hotel_discounts',
    'hotel_discount_amount': 'avg_hotel_discount',
    'cancellation': 'total_cancellations',
    'seats': 'avg_seats',
    'checked_bags': 'avg_checked_bags',
    'return_flight_booked': 'total_return_flights',
    'flight_duration': 'avg_flight_duration_hrs',
    'rooms': 'avg_rooms',
    'nights': 'avg_nights',
    'hotel_per_room_usd': 'avg_usd_per_room_night',
    'geodesic_distance_km': 'avg_geodesic_km',
    'combined_bookings': 'total_combined_bookings',
    'combined_discounts': 'total_combined_discounts'
}, inplace=True)

In [ ]:
# Calculate discount_flight_proportion
basicagg_df['discount_flight_proportion'] = basicagg_df['total_flight_discounts'] / basicagg_df['total_flights_booked']
# Calculate discount_hotel_proportion
basicagg_df['discount_hotel_proportion'] = basicagg_df['total_hotel_discounts'] / basicagg_df['total_hotels_booked']

**Discount Proportions**

**Scaled Average Dollar Saved**

In [ ]:
# Calculate Total USD Saved
basicagg_df['total_usd_saved'] = basicagg_df['avg_flight_discount'] * basicagg_df['total_flights_booked']

# Calculate ADS_per_km
basicagg_df['ADS_per_km'] = basicagg_df['total_usd_saved'] / basicagg_df['avg_geodesic_km']

# Calculate scaled_ADS_per_km
min_ads_per_km = basicagg_df['ADS_per_km'].min()
max_ads_per_km = basicagg_df['ADS_per_km'].max()
basicagg_df['scaled_ADS_per_km'] = (basicagg_df['ADS_per_km'] - min_ads_per_km) / (max_ads_per_km - min_ads_per_km)

**Bargain Hunter Index**

In [ ]:
# Calculate the Bargain Hunter Index
basicagg_df['bargain_hunter_index'] = basicagg_df['scaled_ADS_per_km'] * basicagg_df['discount_flight_proportion'] * basicagg_df['avg_flight_discount']

In [ ]:
aggregated_df = basicagg_df.copy()
aggregated_df.head()

# 4. Segmentation
The most important part of this project will be divided as follows:

**4.1. Data Normalization**
- Clean the aggregated calculations
- Scale the Data: Use feature scaling to normalize the data. This is crucial for distance-based algorithms like k-means.


**4.2. Features EDA**
- Visualize Relationships: Use scatter plots, pair plots, or heatmaps to visualize the relationships between different variables, similar to your teacher's approach. Clean and remove features as needed based on common practices, data insights and domain knowkledge-

**4.3. Number of Clusters Selection**
- Elbow Method: Use the elbow method to find an optimal number of clusters.
- Silhouette Score: Optionally, use the silhouette score for a more in-depth evaluation, especially if computational resources allow.

**4.4 Model Training (Separate Notebook) and Clustering**
- Algorithm Choice: Decide on the segmentation approach and apply it to the TravelTide notebook.

**4.5 Evaluation and Interpretation of clusters**

- Cluster Visualization: Depending on the dimensionality, use scatter plots or pair plots to evaluate the clusters.
- Cluster Profiling: Use descriptive statistics to understand the characteristics of each cluster.

## 4.1 Data Normalization
At this point I'll want to do a checkpoint cleaning of the aggregated data and normalize it to a standard scale to ensure equal weight acroos all features. 

_Post-Note: The separate notebook I ran to compare segmentation approaches (more on that later) also revealed some infinity values. We will handle them now as well._

In [ ]:
# Check for missing values in aggregated_df
null_columns = aggregated_df.columns[aggregated_df.isnull().any()]
print(aggregated_df[null_columns].isnull().sum())

After reseting the kernel and going back to inverstigate it seems that the missing values occur because of a division by zero when calculating some advanced metrics.

For discount proportions: Replacing with the smallest value larger than zero makes sense because it indicates that the proportion of discounted flights (or hotels) is extremely low, almost negligible. We're essentially saying that the user has booked flights (or hotels), but none of them were discounted.

For the rest: These are products of diferent metrics In this context, a missing value likely means that the user did not engage in at least one of the behaviors that would contribute to the behavioural calculation. Therefore, filling it with zero is appropriate, as it indicates a lack of activity.

In [ ]:
# For discount proportions, replace NaN with the smallest value larger than zero in the respective columns
min_discount_flight_proportion = aggregated_df['discount_flight_proportion'].loc[aggregated_df['discount_flight_proportion'] > 0].min()
min_discount_hotel_proportion = aggregated_df['discount_hotel_proportion'].loc[aggregated_df['discount_hotel_proportion'] > 0].min()

aggregated_df['discount_flight_proportion'].fillna(min_discount_flight_proportion, inplace=True)
aggregated_df['discount_hotel_proportion'].fillna(min_discount_hotel_proportion, inplace=True)

# For other calculated metrics, replace NaN with zero
cols_to_fill_zero = ['avg_geodesic_km', 'ADS_per_km', 'scaled_ADS_per_km', 'bargain_hunter_index']
aggregated_df[cols_to_fill_zero] = aggregated_df[cols_to_fill_zero].fillna(0)

# Check if there are still any missing values
aggregated_df.isnull().sum().sum()


Checking for any infinity values and handling them if necessary.

In [ ]:
# Check for infinity values in the aggregated_df
infinity_columns = aggregated_df.columns.to_series()[np.isinf(aggregated_df).any()]

# Display columns that contain infinity values
infinity_columns

As expected the newly created ratios are showing infinity values. We'll take a closer look at the columns `discount_flight_proportion` and `discount_hotel_proportion` to decide the best way to handle the infinite values.

In [ ]:
# Descriptive statistics for the columns with infinite values
columns_with_inf = ['discount_flight_proportion', 'discount_hotel_proportion']
aggregated_df[columns_with_inf].describe()

Given that there is no clear standard deviation and considering the business context and the nature of these metrics, i belive that high values should not be removed as they can be indicative of some customers' particular sensitivity to discounts, which i don't want to ignore at this stage. 

Since these are proportion metrics that would have a max of 1 (if there were no infinites) I will **cap the infinity values at 1**.

In [ ]:
columns_to_cap = ['discount_flight_proportion', 'discount_hotel_proportion']
aggregated_df[columns_to_cap] = np.where(aggregated_df[columns_to_cap] == np.inf, 1, aggregated_df[columns_to_cap])

In [ ]:
# Check again for infinity values
infinity_columns = aggregated_df.columns.to_series()[np.isinf(aggregated_df).any()]

infinity_columns

### Scaling the data 

In this step, we'll scale the data to prepare it for clustering.   
Given the nature of the data in aggregated_df and the business objective of validating Elena's suggested 5 perks, I choose to use Standard Scaling for the following reasons:
- Zero-Centered Data: Standard scaling centers the data around zero and scales it based on the standard deviation. This is particularly useful for algorithms like k-means that calculate distances between data points.
- Interpretable Centroids: Standard scaling will make the cluster centroids more interpretable, which is crucial for understanding the characteristics of each cluster and mapping them to perks.

- Outliers: Your data likely contains outliers, and standard scaling is less sensitive to them compared to Min-Max scaling. This is especially important for k-means clustering, which is sensitive to outliers.

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd  # Importing pandas for DataFrame operations

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the aggregated_df
scaled_df = scaler.fit_transform(aggregated_df)

# Convert the scaled data back to a DataFrame
scaled_df = pd.DataFrame(scaled_df, columns=aggregated_df.columns)

scaled_df.drop(['user_id'], axis=1, inplace=True)
scaled_df.head()

## 4.2 Feature EDA
We will perform feature selection to identify the most relevant features for clustering.
This process can be iterative and flexible. These are the considerations to make when selecting features to cluster:

- Correlation: The idea is to identify highly correlated features so we can consider dropping one from each pair to reduce multicollinearity. However, we don't have to drop them immediately; we can keep them in mind as we proceed with the next steps.

- Feature Importance: We can use machine learning techniques like Random Forest or Gradient Boosting to identify the importance of each feature. This will give us a quantitative measure to validate our choices.

- Domain Knowledge: We'll also consider Elena's suggested perks and see how they align with the features we have. This will help us make informed decisions on which features to keep.

- Dimensionality Reduction (Optional): Depending on the results of the above steps, we may or may not proceed with dimensionality reduction techniques like PCA.

In [ ]:
features_df = scaled_df.copy()
features_df.to_csv('features_df.csv', index=False)

#### Variance Threshold
Before starting with more complex feature selection techniques it is a good idea to first check if there are any metrics with limited variance.  Less likely to contribute significantly to our clustering model they could be dropped straight away.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Initialize VarianceThreshold
selector = VarianceThreshold()

# Fit the features_df to the selector object
selector.fit(features_df)

# Get the variance of each feature
feature_variances = selector.variances_

# Create a DataFrame to show features and their variances
features_variance_df = pd.DataFrame({'Feature': features_df.columns, 'Variance': feature_variances})

# Sort the DataFrame by Variance
features_variance_df = features_variance_df.sort_values(by='Variance', ascending=True)

# Display features with low variance
low_variance_features = features_variance_df[features_variance_df['Variance'] < 0.5]
low_variance_features

It seems we have no features below 0.05 variance, meaning all can potentially be used for clustering

#### Correlation
Generating a heatmap will allow us to detect multicollinearity between two variables, but also decide which high correlations could be useful to merge into new calculated features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = features_df.corr()

# Generate a heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Metrics')
plt.show()

# Filter highly correlated features
highly_correlated = correlation_matrix[(correlation_matrix > 0.8) & (correlation_matrix < 1.0)]

# Create a set to keep track of printed pairs
printed_pairs = set()

# Iterate through the matrix to print the correlated pairs
for row_index, row in highly_correlated.iterrows():
    for col_index, value in row.iteritems():
        if row_index != col_index and not pd.isnull(value):
            # Create a sorted tuple of the pair to avoid duplicates
            pair = tuple(sorted([row_index, col_index]))
            # Check if the pair has been printed before
            if pair not in printed_pairs:
                printed_pairs.add(pair)
                print(f"{row_index} x {col_index} = {value:.2f}")

**Features to keep**:  
1. `avg_clicks_per_session`: Average number of clicks per session, indicating user engagement.
2. `total_cancellations`: Total number of cancellations, indicating indecisiveness or flexibility.
3. `avg_seats`: Average number of seats booked, indicating group size.
4. `avg_checked_bags`: Average number of checked bags, indicating travel needs.
5. `age`: indicating propensity to prefer comfort 
6. `avg_nights`: Average number of nights per hotel stay, indicating trip duration.
7. `avg_geodesic_km`: Average geodesic distance flown, indicating travel distance.
8. `total_combined_bookings`: Total number of bookings for both flights and hotels, indicating preference for package deals.
9. `discount_flight_proportion`: Proportion of flights booked with a discount, indicating sensitivity to flight deals.
10. `discount_hotel_proportion`: Proportion of hotels booked with a discount, indicating sensitivity to hotel deals.
11. `total_combined_discounts`

**Features to drop**: *  
`avg_flight_discount`, `avg_hotel_discount`, `avg_usd_per_room_night`, `total_usd_saved`, `total_flights_booked`, `total_hotels_booked`, `total_flight_discounts`, `total_hotel_discounts`, `total_return_flights`, `scaled_ADS_per_km`, `ADS_per_km`, `bargain_hunter_index`.

*_These features either provide redundant information or don't directly contribute to our goal of customer segmentation for targeted perks._  


In [ ]:
# Create a new DataFrame with only the features to keep
features_to_keep = ['age', 'avg_clicks_per_session', 'total_cancellations', 'avg_seats', 'avg_checked_bags',
                    'avg_geodesic_km', 'total_combined_bookings', 
                    'discount_flight_proportion', 'discount_hotel_proportion', 'total_combined_discounts']

features_df = scaled_df[features_to_keep]

features_df.to_csv('features_df.csv', index=False)
features_df.head()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = features_df.corr()

# Generate a heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Final Features')
plt.show()


## 4.3 Number of Clusters Selection

- Elbow Method: Use the elbow method to find an optimal number of clusters.
- Silhouette Score: Optionally, use the silhouette score for a more in-depth evaluation, especially if computational resources allow.

Algorithm Choice: https://app.noteable.io/published/3bb37272-bc75-41bc-9881-3ad098496b78/TravelTide_notebook_copy_to_decide_on_PCA

On the notebook above i tried different learning models with and wihout PCA transformation. plotted their clusters, clouds, checked silhouette scores... and learned that:  

The high coeficient in Fuzzy Segmentation, shows potential for highly targeted, user-level loyalty marketing. However, since my goal is a straight forward validation of the 5 perks suggested by my Head-of-Marketing, and across-the-board assigning users to one of clearly defined clusters: 

I will opt for **K-means Segmentation** and use PCA only to visualize the clusters in 2D.  

In [ ]:
from sklearn.cluster import KMeans

# Initialize variables
wcss = []  # Within-cluster sum of squares
cluster_range = range(1, 11)  # Testing 1 to 10 clusters

# Fit the k-means model for each number of clusters and calculate WCSS
for i in cluster_range:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(features_df)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method graph
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, wcss, marker='o')
plt.title('Elbow Method For Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

# Initialize an empty list to store silhouette scores
silhouette_scores = []

# Loop through different numbers of clusters
for n_clusters in range(2, 8):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(features_df)
    cluster_labels = kmeans.labels_
    silhouette_avg = silhouette_score(features_df, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f'For n_clusters = {n_clusters}, the silhouette score is {silhouette_avg}.')

The elbow visualization  suggests that 3 or 4 clusters would be optimal, based on the data alone. However, considering Elena's 5 perks, I'm confident that the silhouette score for K=5 indicates that the clusters spread is not too diferent from 3 or 4 clusters, especially after I iterated PCA vizz with all 3 options.  
Thus, **going with k=5 is a good balance between the data insights and the business objectives.**

## 4.4 Clustering



**Applying K-means Clustering**


In [ ]:
from sklearn.cluster import KMeans

# Initialize K-means with 5 clusters
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
# Fit the model
kmeans.fit(features_df)
# Get the cluster labels
cluster_labels = kmeans.labels_
# Add the cluster labels to the original data
features_df['Cluster'] = cluster_labels

## 4.5 Evaluation and Interpretation
In this step, I'll evaluate and interpret the clusters formed. I'll visualize the clusters and profile them using descriptive statistics.

In [ ]:
# Calculate the mean value for each cluster
cluster_summary = features_df.groupby('Cluster').mean().reset_index()

# Plot the heatmap for cluster centroids
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_summary.drop('Cluster', axis=1).transpose(), annot=True, cmap='coolwarm', cbar=True, xticklabels=cluster_summary['Cluster'], yticklabels=cluster_summary.columns[1:])
plt.title('Cluster Centroids')
plt.show()


**Cluster 0:**
Older age group (age: 0.59)

Low clicks per session (avg_clicks_per_session: -0.42)  
Low cancellations (total_cancellations: -0.32)
Fewer seats and bags (avg_seats: -0.52, avg_checked_bags: -0.45)  
Average geodesic distance (avg_geodesic_km: -0.025)  
Low combined bookings (total_combined_bookings: -0.48)  
Possible Perk: **Free Hotel Meal**

**Cluster 1:**
Very high clicks per session (avg_clicks_per_session: 2)
Very high cancellations (total_cancellations: 3)

Younger age group (age: 0.065)  
Average seats and bags (avg_seats: 0.97, avg_checked_bags: 0.8)  
High geodesic distance (avg_geodesic_km: 0.095)  
Possible Perk: **No Cancellation Fees**

**Cluster 2:** High discount proportions)

Middle-aged (age: 0.11)  
Low clicks per session (avg_clicks_per_session: -0.29)  
Low cancellations (total_cancellations: -0.32)  
Possible Perk: **Exclusive Discounts**

**Cluster 3:** High combined bookings (total_combined_bookings: 0.96)

Middle-aged (age: 0.12)  
Average clicks per session (avg_clicks_per_session: 0.17)  
Low cancellations (total_cancellations: -0.33)
Very Low discounts  
Possible Perk: **Book a Flight Get 1 Night Free**

Cluster 4:

Very young or outliers (age: -1.3)  
Very low clicks per session (avg_clicks_per_session: -0.63)  
Low cancellations (total_cancellations: -0.29)  
Very low seats and bags (avg_seats: -1, avg_checked_bags: -0.74)  
Very low geodesic distance (avg_geodesic_km: -0.59)  
Possible Perk: **This cluster seems to be an outlier or a very specific group. It might not be suitable for any of the perks.**

### On the Absence of a "Free Checked Bags" Cluster

After several iterations of diferent clustering models, it became evident that none of the clusters showed a strong inclination towards the need for checked bags. While some clusters did have higher values for the `avg_checked_bags` metric, other behaviors clearly dominated those clusters, overshadowing the need for free checked bags.

This suggests that the need for checked bags is either uniformly distributed across all types of customers, not a defining feature for any specific customer segment, or overshadowed by other more dominant behaviors in our dataset.




# 5. Perk Allocation
Now, I'll assign each user to a perk based on their most characteristic behaviour(s).

### Adding Cluster Labels to Original Data
I will add the cluster labels to `features_df` where I performed the K-means clustering. Then, I merged this back to `aggregated_df` to align each `user_id` with its respective cluster. This will help in allocating perks to individual users.


In [ ]:
# Add the cluster labels to features_df
features_df['Cluster'] = cluster_labels

# Merge this back to aggregated_df to get user_id and Cluster in the same DataFrame
final_df = pd.merge(aggregated_df, features_df[['Cluster']], left_index=True, right_index=True)

# Now, each user_id in the final_df has a Cluster label
final_df.head()

### Allocation
Now that each user is assigned to a cluster, the next step is to allocate perks based on the most defining characteristics of each cluster. This will allow us to offer targeted perks that are most likely to engage the specific types of users in each cluster.


In [ ]:
# Define a function to map clusters to perks
def allocate_perk(cluster):
    if cluster == 0:
        return "Free Hotel Meal"
    elif cluster == 1:
        return "No Cancellation Fees"
    elif cluster == 2:
        return "Exclusive Discounts"
    elif cluster == 3:
        return "Book a Flight, Get 1 Night Free"
    else:
        return "No Perk"

# Apply the function to allocate perks
final_df['Allocated_Perk'] = final_df['Cluster'].apply(allocate_perk)

# Show the first few rows to verify
final_df.head()

# 6. Validation and Evaluation
In this final step of this notebook, I'll validate the clusters formed and evaluate how well they align with the allocated perks. This will involve statistical tests to ensure the clusters are significantly different and an analysis to confirm that the perks allocated are suitable for each cluster.

### Statistical Tests for Validation
To validate the effectiveness of our clustering and perk allocation, we'll perform some statistical tests. This will help us understand how well our clusters are separated and if the perks allocated are meaningful.

In [ ]:
# Importing necessary libraries for statistical tests
from scipy.stats import chi2_contingency, ttest_ind

### Validation and Evaluation: Statistical Tests
I'll run some statistical tests to validate the clusters. This will help ensure that the clusters are statistically significant and not formed by random chance.

**Chi-Square Test**

I'm running a Chi-Square test to validate the clusters based on the allocated perks. This will help me understand if the clusters are statistically significant.  
If the p-value is less than 0.05, it would generally mean that the clusters are statistically significant.

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table for 'Cluster' and 'Allocated_Perk'
contingency_table = pd.crosstab(final_df['Cluster'], final_df['Allocated_Perk'])

# Perform tbhe Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the p-value
print(f'Chi-Square Test p-value: {p}')

 A p-value of 0.0 generally indicates that the clusters are statistically significant, meaning that the differences between them are not due to random chance. This adds a layer of confidence to my clustering and perk allocation strategy.

# 7. Conclusion

I have written a non-technical report for the marketing and executive teams which can be consulted here: 